In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as npr
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from keras.optimizers import Adam
from keras_nlp.layers import PositionEmbedding

In [3]:
seed = 428

np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)

In [4]:
def get_masked_input_and_labels(encoded_texts, n_cat):
    # For each sentence, mask each word one-by-one

    encoded_texts_masked = []
    y_labels = []

    for encoded_text in encoded_texts:
        for i in range(len(encoded_text)):
            encoded_text_masked = np.copy(encoded_text)
            y_label = encoded_text_masked[i]
            encoded_texts_masked.append(np.delete(encoded_text_masked, i))
            y_labels.append(np.array([y_label]))

    return np.array(encoded_texts_masked), np.array(y_labels)

In [5]:
#### K = number of countries = number of capitals = number of universities = number of mascots
#### M = number of words only used by each topic
#### S = number of words used by both topics
#### L = sentence length
#### q1, q2 = probability of having 1 or 2 pairs
#### embed_dim = dimension of embeddings
#### n_sentences = number of training sentences

def train_model(K, M, S, L, q1, q2, embed_dim, n_sentences):
    
    countries = ['country_' + str(i) for i in range(K)]
    capitals = ['capital_' + str(i) for i in range(K)]
    universities = ['university_' + str(i) for i in range(K)]
    mascots = ['mascot_' + str(i) for i in range(K)]
    random_capitals = ['random_capital_' + str(i) for i in range(M)]
    random_mascots = ['random_mascot_' + str(i) for i in range(M)]
    randoms = ['random_' + str(i) for i in range(S)]

    vocabs = countries + capitals + universities + mascots + random_capitals + random_mascots + randoms
    vocab_map = {}

    for i in range(len(vocabs)):
        vocab_map[vocabs[i]] = i
        
    sentences = []
    sentences_number = []
    
    q0 = 1 - q1 - q2

    for i in range(n_sentences):

        sentence = []
        
        temp = npr.uniform()
        temp2 = npr.uniform()
        
        if temp2 <= q0:
            n_pairs = 0
        elif temp2 <= q0 + q1:
            n_pairs = 1
        else:
            n_pairs = 2
        
        if temp <= 0.5: ### country - capital
        
            pairs = np.random.choice(np.arange(K), n_pairs, replace = False)
#             pairs2 = np.random.choice(np.arange(M), n_pairs, replace = False)
            for pair in pairs:
                sentence.append(countries[pair])
                sentence.append(capitals[pair])        
#             for pair in pairs2:
#                 temp3 = npr.uniform()
#                 if temp3 <= 0.5:
#                     sentence.append(random_capitals[pair])
#                 else:
#                     sentence.append(random_mascots[pair])
            
            randoms_dup = 4 * random_capitals + 2 * randoms + 1 * random_mascots
            sentence += list(np.random.choice(randoms_dup, L - 2 * n_pairs, replace = False))  
                 
        else: ### university - mascot
            
            pairs = np.random.choice(np.arange(K), n_pairs, replace = False)
#             pairs2 = np.random.choice(np.arange(M), n_pairs, replace = False)
            for pair in pairs:
                sentence.append(universities[pair])
                sentence.append(mascots[pair])        
#             for pair in pairs2:
#                 temp3 = npr.uniform()
#                 if temp3 <= 0.5:
#                     sentence.append(random_mascots[pair])
#                 else:
#                     sentence.append(random_capitals[pair])

            
            randoms_dup = 1 * random_capitals + 2 * randoms + 4 * random_mascots
            sentence += list(np.random.choice(randoms_dup, L - 2 * n_pairs, replace = False))  
            
        
        

        #sentence += list(np.random.choice(randoms + random_capitals + random_mascots, L - 2 * n_pairs, replace = False))  

        sentence_number = [vocab_map[i] for i in sentence]
        sentences.append(sentence)
        sentences_number.append(sentence_number)
        
    x_train = np.array(sentences_number)
    n_cat = len(vocab_map)
    x_masked_train, y_masked_labels_train = get_masked_input_and_labels(x_train, n_cat)
    
    callback = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)
    inputs = layers.Input((x_masked_train.shape[1],), dtype=tf.int64)
    word_embeddings = layers.Embedding(n_cat, embed_dim, name="word_embedding")(inputs)
    encoder_output = layers.GlobalAveragePooling1D()(word_embeddings)
    mlm_output = layers.Dense(n_cat, name="mlm_cls", activation="softmax", use_bias=False)(encoder_output)
    mlm_model = keras.Model(inputs = inputs, outputs = mlm_output)
    adam = Adam()
    mlm_model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)

    history = mlm_model.fit(x_masked_train, y_masked_labels_train,
                        validation_split = 0.5, callbacks = [callback], 
                        epochs=500, batch_size=128, verbose=0)
    
    return sentences, vocab_map, mlm_model

In [6]:
def get_acc_prob(K, M, S, L, q1, q2, embed_dim, n_sentences, n_samples):
    
    sentences, vocab_map, current_model = train_model(K, M, S, L, q1, q2, embed_dim, n_sentences)

    acc_capitals = []
    prob_capitals = []

    for _ in range(n_samples):
        sentence = []
        random_capitals = np.random.choice(np.arange(K), int(L/2), replace = False)
        for random_capital in random_capitals:
            sentence.append('country_' + str(random_capital))
            sentence.append('capital_' + str(random_capital))
        sentence = sentence[:-1]
        sentence_number = [vocab_map[i] for i in sentence]
        temp = keras.backend.function(inputs = current_model.layers[0].input, outputs = current_model.layers[-1].output) \
            (np.array(sentence_number).reshape(1,len(sentence_number)))
        actual = vocab_map['capital_' + str(random_capitals[-1])]
        acc_capitals.append(1 if np.argsort(-1 * temp)[0][0] == actual else 0)
        prob_capitals.append(temp[0][vocab_map['capital_' + str(random_capitals[-1])]])
        
    acc_mascots = []
    prob_mascots = []

    for _ in range(n_samples):
        sentence = []
        random_mascots = np.random.choice(np.arange(K), int(L/2), replace = False)
        for random_mascot in random_mascots:
            sentence.append('university_' + str(random_mascot))
            sentence.append('mascot_' + str(random_mascot))
        sentence = sentence[:-1]
        sentence_number = [vocab_map[i] for i in sentence]
        temp = keras.backend.function(inputs = current_model.layers[0].input, outputs = current_model.layers[-1].output) \
            (np.array(sentence_number).reshape(1,len(sentence_number)))
        actual = vocab_map['mascot_' + str(random_mascots[-1])]
        acc_mascots.append(1 if np.argsort(-1 * temp)[0][0] == actual else 0)
        prob_mascots.append(temp[0][vocab_map['mascot_' + str(random_mascots[-1])]])
        

    return sentences, current_model, vocab_map, (np.mean(acc_capitals), np.mean(prob_capitals)), \
                (np.mean(acc_mascots), np.mean(prob_mascots))

In [7]:
K = 10 # number of countries
L = 8 # sentence length
M = 20 # number of words used by each topic
S = 20 # number of words used by both topics
embed_dim = 10 # CBOW embedding dimension
n_sentences = 50000 # number of sentences in the training set
n_samples = 1000

In [21]:
q0 = 0 # probability of having 0 pairs
q1 = 1 # probability of having 1 pair
q2 = 0 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, M, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(0.0, 4.308565e-26)
(0.0, 1.3986933e-25)
(0.0, 1.8090795e-24)
(0.0, 4.942542e-28)
(0.0, 1.0876332e-29)
(0.0, 2.3819443e-29)
(0.0, 2.453412e-26)
(0.0, 4.064949e-27)
(0.0, 1.1428793e-31)
(0.0, 3.1511555e-31)
(0.0, 3.1780113e-32)
(0.0, 3.857537e-30)
(0.0, 8.7947545e-26)
(0.0, 4.918049e-25)
(0.0, 3.9887345e-32)
(0.0, 2.8513771e-31)
(0.0, 3.3897772e-34)
(0.0, 5.3152313e-33)
(0.0, 4.2526162e-24)
(0.0, 2.0675768e-28)
(0.0, 6.217274122795354e-25)
(0.0, 6.364684503165447e-26)


In [22]:
q0 = 0 # probability of having 0 pairs
q1 = 0 # probability of having 1 pair
q2 = 1 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, M, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(0.0, 3.04641e-05)
(0.004, 0.0014195961)
(0.1, 0.01512116)
(0.186, 0.029488053)
(0.0, 9.938566e-06)
(0.0, 2.2380182e-05)
(0.268, 0.099668704)
(0.347, 0.123830736)
(0.484, 0.10750331)
(0.008, 0.0031037098)
(0.0, 9.88072e-05)
(0.002, 0.0018419767)
(0.0, 0.00012829478)
(0.0, 0.00012847265)
(0.555, 0.2068598)
(0.454, 0.19144528)
(0.106, 0.02168327)
(0.38, 0.063758)
(0.049, 0.012646965)
(0.0, 3.684311e-05)
(0.15619999999999998, 0.04637507106390331)
(0.1381, 0.041507504424407676)


In [23]:
q0 = 1/2 # probability of having 0 pairs
q1 = 1/2 # probability of having 1 pair
q2 = 0 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, M, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(0.272, 0.18631977)
(0.161, 0.09608693)
(0.283, 0.17925784)
(0.315, 0.21639058)
(0.259, 0.16353607)
(0.248, 0.16416918)
(0.163, 0.0949261)
(0.496, 0.35963252)
(0.344, 0.26159865)
(0.259, 0.17734715)
(0.18, 0.13733216)
(0.199, 0.14726505)
(0.471, 0.33643916)
(0.263, 0.1592943)
(0.248, 0.1677058)
(0.364, 0.21958761)
(0.274, 0.19665843)
(0.217, 0.14915676)
(0.28, 0.17286813)
(0.211, 0.12408617)
(0.2774, 0.18966421112418175)
(0.2733, 0.18130162581801412)


In [24]:
q0 = 1/2 # probability of having 0 pairs
q1 = 0 # probability of having 1 pair
q2 = 1/2 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, M, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(1.0, 0.9998805)
(1.0, 0.9997428)
(1.0, 0.99991995)
(1.0, 0.99981165)
(1.0, 0.9998041)
(1.0, 0.9998212)
(1.0, 0.9999597)
(1.0, 0.9999724)
(1.0, 0.99983376)
(1.0, 0.9999105)
(1.0, 0.99993104)
(1.0, 0.9998866)
(1.0, 0.9999068)
(1.0, 0.99989814)
(1.0, 0.9999528)
(1.0, 0.99996865)
(1.0, 0.99989855)
(1.0, 0.9999031)
(1.0, 0.99996424)
(1.0, 0.9999544)
(0.9999999999999999, 0.99990513920784)
(0.9999999999999999, 0.9998869359493255)


In [9]:
q0 = 0 # probability of having 0 pairs
q1 = 1/2 # probability of having 1 pair
q2 = 1/2 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, M, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(1.0, 0.99772245)
(1.0, 0.9950919)
(1.0, 0.99523866)
(1.0, 0.99691105)
(1.0, 0.9974566)
(1.0, 0.9967803)
(1.0, 0.99286866)
(1.0, 0.9958752)
(1.0, 0.99812424)
(1.0, 0.99575967)
(1.0, 0.99908465)
(1.0, 0.99904484)
(1.0, 0.9934894)
(1.0, 0.99956924)
(1.0, 0.9980884)
(1.0, 0.99824464)
(1.0, 0.9978714)
(1.0, 0.99777293)
(1.0, 0.99634326)
(1.0, 0.9982415)
(0.9999999999999999, 0.9966287732124329)
(0.9999999999999999, 0.9973291218280792)


In [10]:
q0 = 1/3 # probability of having 0 pairs
q1 = 1/3 # probability of having 1 pair
q2 = 1/3 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, M, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(1.0, 0.99972963)
(1.0, 0.9996282)
(1.0, 0.99952465)
(1.0, 0.99956495)
(1.0, 0.9996334)
(1.0, 0.99982065)
(1.0, 0.99957174)
(1.0, 0.9994311)
(1.0, 0.999811)
(1.0, 0.9994858)
(1.0, 0.99963486)
(1.0, 0.9994331)
(1.0, 0.99978125)
(1.0, 0.9999253)
(1.0, 0.99969566)
(1.0, 0.9994158)
(1.0, 0.9997359)
(1.0, 0.99883)
(1.0, 0.9998027)
(1.0, 0.99984115)
(0.9999999999999999, 0.9996920764446258)
(0.9999999999999999, 0.999537605047226)
